**Engy-4390** Nuclear Systems Design and Analysis, UMass Lowell, Chem. Eng. Nuclear Program Fall 2024

# Uranium Mill Network Dynamics Simulation Report
$
  \newcommand{\Amtrx}{\boldsymbol{\mathsf{A}}}
  \newcommand{\Bmtrx}{\boldsymbol{\mathsf{B}}}
  \newcommand{\Mmtrx}{\boldsymbol{\mathsf{M}}}
  \newcommand{\Imtrx}{\boldsymbol{\mathsf{I}}}
  \newcommand{\Pmtrx}{\boldsymbol{\mathsf{P}}}
  \newcommand{\Lmtrx}{\boldsymbol{\mathsf{L}}}
  \newcommand{\Umtrx}{\boldsymbol{\mathsf{U}}}
  \newcommand{\Smtrx}{\boldsymbol{\mathsf{S}}}
  \newcommand{\xvec}{\boldsymbol{\mathsf{x}}}
  \newcommand{\uvar}{\boldsymbol{u}}
  \newcommand{\fvar}{\boldsymbol{f}}
  \newcommand{\avec}{\boldsymbol{\mathsf{a}}}
  \newcommand{\bvec}{\boldsymbol{\mathsf{b}}}
  \newcommand{\cvec}{\boldsymbol{\mathsf{c}}}
  \newcommand{\rvec}{\boldsymbol{\mathsf{r}}}
  \newcommand{\mvec}{\boldsymbol{\mathsf{m}}}
  \newcommand{\gvec}{\boldsymbol{\mathsf{g}}}
  \newcommand{\zerovec}{\boldsymbol{\mathsf{0}}}
  \newcommand{\norm}[1]{\bigl\lVert{#1}\bigr\rVert}
  \newcommand{\transpose}[1]{{#1}^\top}
  \DeclareMathOperator{\rank}{rank}
  \newcommand{\Power}{\mathcal{P}}
$

**Student(s):** Alyssa Puglisi <br>
**Course:** Engy-4390 Nuclear Systems Design and Analysis  <br>
**Mentor:** Prof. Valmor F. de Almeida

UMass Lowell, Dept. of Chemical Engineering, Nuclear Energy Program 
<br>
**16Oct2024**

[Project Repository on `GitHub`](https://github.com/dpploy/engy-4390/tree/main/projects/2024/u-mill)

---
## Table of Contents<a id="toc"></a>
* [Problem Statment](#prob)
* [Introduction](#intro)
* [Objectives](#obj)
* [Tools](#tools)
* [Approach and Dynamics Equations](#app)
* [Challenges & Course Corrections](#correct)
* [Initialization of Solvent Extraction Module](#IPM)
* [Startup](#su)
* [Steady-State](#ss)
* [Disturbance](#malfunction)
* [References](#refs)
---

In [17]:
from cortix import ReactionMechanism

## [Problem Statement](#toc)<a id="prob"></a>

Design a dynamic system that accurately models the reactive mass transfer with multiple phases occuring in the solvent extraction process during White Mesa's uranium milling. 

Mechanism of Extraction & Stripping:

In [24]:
rxn_mech = ReactionMechanism(file_name='solvex-stripping-rxn-mechanism.txt', order_species=True)
rxn_mech.md_print()
#TODO C11H20O2 is only showing up as C1 H2 O2 ??

8 **Species:** 
 ${\mathrm{H}_{2}\mathrm{S}\mathrm{O}_{4}}_{\mathrm{(a)}} \quad  {\mathrm{H}\mathrm{S}\mathrm{O}_{4}^-}_\mathrm{(a)} \quad  {\mathrm{S}\mathrm{O}_{4}^-}_{\mathrm{(a)}} \quad  {\mathrm{U}\mathrm{O}_{2}[\mathrm{S}\mathrm{O}_{4}]_{2}^-}_{\mathrm{(a)}} \quad  {[\mathrm{C}_{1}\mathrm{H}_{2}\mathrm{O}_{2}]_{3}\mathrm{N}}_\mathrm{(o)} \quad  {[\mathrm{C}_{1}\mathrm{H}_{2}\mathrm{O}_{2}]_{3}\mathrm{N}\mathrm{H}\bullet\mathrm{H}\mathrm{S}\mathrm{O}_{4}}_{\mathrm{(o)}} \quad  {[[\mathrm{C}_{1}\mathrm{H}_{2}\mathrm{O}_{2}]_{3}\mathrm{N}\mathrm{H}]_{2}\mathrm{S}\mathrm{O}_{4}}_{\mathrm{(o)}} \quad  {[[\mathrm{C}_{1}\mathrm{H}_{2}\mathrm{O}_{2}]_{3}\mathrm{N}\mathrm{H}]_{2}\mathrm{U}\mathrm{O}_{2}[\mathrm{S}\mathrm{O}_{4}]_{2}}_{\mathrm{(o)}}$

4 **Reactions:** 
 \begin{align*} 
{\mathrm{H}_{2}\mathrm{S}\mathrm{O}_{4}}_{\mathrm{(a)}}\ + \ 2.0\,{[\mathrm{C}_{1}\mathrm{H}_{2}\mathrm{O}_{2}]_{3}\mathrm{N}}_\mathrm{(o)}\ &\longleftrightarrow \ {[[\mathrm{C}_{1}\mathrm{H}_{2}\mathrm{O}_{2}]_{3}\mathrm{N}\mathrm{H}]_{2}\mathrm{S}\mathrm{O}_{4}}_{\mathrm{(o)}}\\ 
{\mathrm{H}_{2}\mathrm{S}\mathrm{O}_{4}}_{\mathrm{(a)}}\ + \ {[\mathrm{C}_{1}\mathrm{H}_{2}\mathrm{O}_{2}]_{3}\mathrm{N}}_\mathrm{(o)}\ &\longleftrightarrow \ {[\mathrm{C}_{1}\mathrm{H}_{2}\mathrm{O}_{2}]_{3}\mathrm{N}\mathrm{H}\bullet\mathrm{H}\mathrm{S}\mathrm{O}_{4}}_{\mathrm{(o)}}\\ 
{\mathrm{U}\mathrm{O}_{2}[\mathrm{S}\mathrm{O}_{4}]_{2}^-}_{\mathrm{(a)}}\ + \ {[[\mathrm{C}_{1}\mathrm{H}_{2}\mathrm{O}_{2}]_{3}\mathrm{N}\mathrm{H}]_{2}\mathrm{S}\mathrm{O}_{4}}_{\mathrm{(o)}}\ &\longleftrightarrow \ {\mathrm{S}\mathrm{O}_{4}^-}_{\mathrm{(a)}}\ + \ {[[\mathrm{C}_{1}\mathrm{H}_{2}\mathrm{O}_{2}]_{3}\mathrm{N}\mathrm{H}]_{2}\mathrm{U}\mathrm{O}_{2}[\mathrm{S}\mathrm{O}_{4}]_{2}}_{\mathrm{(o)}}\\ 
{\mathrm{U}\mathrm{O}_{2}[\mathrm{S}\mathrm{O}_{4}]_{2}^-}_{\mathrm{(a)}}\ + \ 2.0\,{[[\mathrm{C}_{1}\mathrm{H}_{2}\mathrm{O}_{2}]_{3}\mathrm{N}\mathrm{H}]_{2}\mathrm{S}\mathrm{O}_{4}}_{\mathrm{(o)}}\ &\longleftrightarrow \ 2.0\,{\mathrm{H}\mathrm{S}\mathrm{O}_{4}^-}_\mathrm{(a)}\ + \ {[[\mathrm{C}_{1}\mathrm{H}_{2}\mathrm{O}_{2}]_{3}\mathrm{N}\mathrm{H}]_{2}\mathrm{U}\mathrm{O}_{2}[\mathrm{S}\mathrm{O}_{4}]_{2}}_{\mathrm{(o)}}\\ 
\end{align*} 


The amine, on contact with a sulphuric acid solution, is converted into the sulphate or bisulphate salt.

The actual extraction reactions are complex. However, in a simplified way, the extraction can be depicted as an anion exchange where the anionic uranium complex from the aqueous solution replaces the sulphate or bisulphate ion in the organic phase. 

## [Introduction](#toc)<a id="intro"></a>


Solvent extraction, also called liquid-liquid equilibrium or liquid-liquid extraction, is a method used to separate a compound from two immiscible liquids. These two liquids are typically denoted as an aqeuous phase and an organic phase. 

Extraction is performed based on the relative solubility of the solute in the two phases with the driving force being chemical potential. The resulting products will be more stable than the initial solution. This separation process is commonly preformed in separatory funnels or in countercurrrent mizer settler units. 

The uranium solvent extraction circuit consists of:

(a) Four extraction stages, 

(b) One organic washing stage,

(c) Four stripping stages, 

(d) One organic regeneration stage.

## [Objectives](#toc)<a id="obj"></a>

## [Tools](#toc)<a id="tools"></a>

## [Approach and Dynamics Equations](#toc)<a id="app"></a>

## [Challenges & Course Correctons](#toc)<a id="correct"></a>

## [Initialization of Solvent Extraction Modules](#toc)<a id="IPM"></a>

```python   
                                      |
                                      |  Extraction Feed (from Decantation-Filtration filtrate)
                                      |
                                      v
                              |----------------|
                              |                |
         Organic Feed ------->|                |------> Organic Product (to Scrubbing internal)
         (internal source)    |    Solvent     |
                              |   Extraction   |
         Raffinate <----------|                |<------ Scrub Raffinate (from Scrubbing internal)
         Stream (to CCD Bank) |                |
                              |----------------|
          Organic Product <---|                |<------ Organic Feed (Organic Product from Solv. Extr. internal)
          (to Strip internal) |                |
                              |   Scrubbing    |
           Scrub Feed ------->|                |-------> Scrub Raffinate (to Solvent Extraction internal)
           (internal source)  |                |
                              |----------------|
          Organic Feed ------>|                |-------> Organic Regeneration (to Solvent Extraction not done)
        (from Scrub internal) |   Stripping    |
                              |                |<------ Stripping Feed (internal source)
                              |________________|<------ Stripping Feed (from Precipitation not implemented)
                                      |
                                      |
                                      |
                                      v
                               Stripping Product (Precipitation feed)
```

## [References](#toc)<a id="refs"></a>

- [1] “35 No. 9 Uranium Extraction Technology - IAEA.” Uranium Extraction Technology, IAEA, 1993, https://www-pub.iaea.org/MTCD/Publications/PDF/trs359_web.pdf. 
- [2] "31 No. 3 Manual on Laboratory Testing for Uranium Ore Precessing - IAEA." Manual on Laboratory Testing for Uranium Ore Precessing, IAEA, 1990, https://www-pub.iaea.org/MTCD/Publications/PDF/trs313_web.pdf